### Homework
1) Write your own function to compute the **confusion matrix** and the **diagonal** with the classification scores for each class.

In [ ]:
def compute_confusion_matrix(test_y, pred_y):
  classes = np.unique(test_y)
  confmat = np.zeros((len(classes), len(classes)))
  for i in range(len(classes)):
    for j in range(len(classes)):
       confmat[i, j] = np.sum((test_y == classes[i]) & (pred_y == classes[j]))

  return confmat, (confmat.diagonal() / confmat.sum(axis=1))

2) Download the CIFAR 10 dataset from Virtuale (`CIFAR-10-simple.zip`).
Load the dataset and the classes, try to solve the classification problem. You can use only the data provided in the `train` folder to train your model.
Compute the final accuracy with the folder `test`.

CIFAR 10 (https://www.cs.toronto.edu/~kriz/cifar.html) consists in **32x32 colour images** (RGB) divided in **10 classes**. There are **300 samples in training** and **50 testing samples** for each class.

Upload your result (`results.txt`) on Virtuale, including the classifier(s) and features used. The student with best accuracy will present his/her solution in the next lecture!


In [ ]:
import time
import os
import sklearn
import numpy as np
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
from glob import glob
from os.path import join
import cv2
from skimage.feature import hog, local_binary_pattern
from tqdm import tqdm


In [ ]:
np.random.seed(1821)

Load data

In [ ]:
!unzip -q CIFAR-10-simple.zip -d /content

In [ ]:
dataset_path_test = '/content/CIFAR-10-simple/test'
dataset_path_train = '/content/CIFAR-10-simple/train'

test_set = glob(join(dataset_path_test, '*', '*.jpg'))
train_set = glob(join(dataset_path_train, '*', '*.jpg'))

print('Test set: ', len(test_set))
print('Train set: ', len(train_set))

Test set:  500
Train set:  3000


Labels:
- Airplane: 0
- Automobile: 1
- Bird: 2
- Cat: 3
- Deer: 4
- Dog: 5
- Frog: 6
- Horse: 7
- Ship: 8
- Truck 9

In [ ]:
def get_labels(image):
  if 'airplane' in image:
    return 0
  elif 'automobile' in image:
    return 1
  elif 'bird' in image:
    return 2
  elif 'cat' in image:
    return 3
  elif 'deer' in image:
    return 4
  elif 'dog' in image:
    return 5
  elif 'frog' in image:
    return 6
  elif 'horse' in image:
    return 7
  elif 'ship' in image:
    return 8
  elif 'truck' in image:
    return 9
  else:
    raise NotImplementedError('Not existing class!')

Extract Features with BW images

In [ ]:
def extract_features(images, feat_type, img_size):

    labels = []
    features = []

    for image in tqdm(images):

        # open the image
        img = cv2.imread(image, 0)

        # resize the image
        img = cv2.resize(img, (img_size, img_size))

        # compute the features
        if feat_type == 'hog':
            feat = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2))
        elif feat_type == 'lbp':
            feat = np.ravel(local_binary_pattern(img, P=100, R=5))
        elif feat_type == 'img':
            img = img / 256.0
            feat = np.ravel(img)
        else:
            raise NotImplementedError('Not implemented feature!')

        # append features and labels
        features.append(feat)
        labels.append(get_labels(image))

    return features, labels

### Sample Distribution:
- Train: 76%
- Test: 14%
- Validation: 10%


In [ ]:
np.random.shuffle(test_set)
np.random.shuffle(train_set)
trainset = train_set[350:]
valset = train_set[0:350]
testset = test_set
print('Total: {} splitted in Train: {}, Val: {} and Test: {}'.format(len(train_set) + len(test_set), len(trainset), len(valset), len(testset)))

Total: 3500 splitted in Train: 2650, Val: 350 and Test: 500


In [ ]:
size = [
    32,
    64,
    128
]

features = [
    'hog',
    'lbp',
    'img'
]

for size in size:
  for feature in features:
    t1 = time.time()
    train_x, train_y = extract_features(trainset, feature, size)
    val_x, val_y = extract_features(valset, feature, size)
    test_x, test_y = extract_features(testset, feature, size)
    clf = svm.SVC(gamma=0.001, C=100., kernel='rbf', verbose=False)
    clf.fit(train_x, train_y)
    clf.score(val_x, val_y)
    y_pred = clf.predict(test_x)
    t2 = time.time()
    print()
    print('Final Accuracy {} {}: {:.3f} - Elapsed Time: {}'.format((feature),(size),(accuracy_score(test_y, y_pred)),(t2 - t1)))
    print()

100%|██████████| 500/500 [00:00<00:00, 1458.59it/s]



Final Accuracy hog 32: 0.410 - Elapsed Time: 4.562856435775757



100%|██████████| 500/500 [00:01<00:00, 424.40it/s]



Final Accuracy lbp 32: 0.100 - Elapsed Time: 15.548899412155151



100%|██████████| 500/500 [00:00<00:00, 16863.29it/s]


...


Final Accuracy hog 128: 0.404 - Elapsed Time: 111.34847807884216



100%|██████████| 500/500 [00:16<00:00, 29.98it/s]



Final Accuracy lbp 128: 0.100 - Elapsed Time: 340.19195008277893



100%|██████████| 500/500 [00:00<00:00, 7916.35it/s]



Final Accuracy img 128: 0.278 - Elapsed Time: 200.0074052810669



HOG seems to be the most accurate feature descriptor (compared to LBP and np.ravel), therefore, we will check the accuracy of HOG using different classifiers:

- SVC
- Random Forest
- Ada Boost
- Decision Tree
- Quadratic Discrimination

In [ ]:
def apply_hog(images, img_size):

    labels = []
    features = []

    for image in (images):

        # open the image
        img = cv2.imread(image, 0)

        # resize the image
        img = cv2.resize(img, (img_size, img_size))

        # compute the feature
        feat = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2))

        # append features and labels
        features.append(feat)
        labels.append(get_labels(image))

    return features, labels

In [ ]:
from numpy.core.arrayprint import format_float_scientific
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
size = [
    32,
    64,
    128
]

names = [
    "SVC",
    "Random Forest",
    "Ada Boost",
    "Decision Tree",
    "Quadratic Discrimination"
]

classifiers = [
    svm.SVC(gamma=0.001, C=100., kernel='rbf', verbose=False, probability=False),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    DecisionTreeClassifier(),
    QuadraticDiscriminantAnalysis()
]

for size in size:
  for name, clf in zip(names, classifiers):
    t1 = time.time()
    train_x, train_y = apply_hog(trainset, size)
    val_x, val_y = apply_hog(valset, size)
    test_x, test_y = apply_hog(testset, size)
    clf.fit(train_x, train_y)
    clf.score(val_x, val_y)
    y_pred = clf.predict(test_x)
    t2 = time.time()
    print()
    print('Elapsed Time: {}'.format((t2 - t1)))
    print('Final accuracy: {} {}, {:.3f}'.format((name),(size),(accuracy_score(test_y, y_pred))))   
    print()

100%|██████████| 500/500 [00:00<00:00, 1345.59it/s]



Elapsed Time: 4.527556419372559
Final accuracy: SVC 32, 0.410



100%|██████████| 500/500 [00:00<00:00, 1488.21it/s]



Elapsed Time: 2.461017370223999
Final accuracy: Random Forest 32, 0.258



100%|██████████| 500/500 [00:00<00:00, 1417.68it/s]


...


Elapsed Time: 201.02255296707153
Final accuracy: Ada Boost 128, 0.266



100%|██████████| 500/500 [00:04<00:00, 104.65it/s]



Elapsed Time: 61.4078905582428
Final accuracy: Decision Tree 128, 0.190



100%|██████████| 500/500 [00:04<00:00, 104.85it/s]
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")



Elapsed Time: 41.280163049697876
Final accuracy: Quadratic Discrimination 128, 0.080



The **Decision Tree** classifier and the **Quadratic Discrimination** classifier provide the lowest accuracy score.

On the other hand, **SVC** is the most accurate, followed by **Ada Boost** and **Random Forest**.

Now, we will try to optimize these classifiers:

In [ ]:
size = [
    32,
    64,
    128
]

names = [
    "SVC",
    "Random Forest",
    "Ada Boost"
]

classifiers = [
    svm.SVC(gamma='scale', C=20., kernel='rbf', verbose=True, probability=False),
    RandomForestClassifier(n_estimators=500),
    AdaBoostClassifier(n_estimators=1000),
]

for size in size:
  for name, clf in zip(names, classifiers):
    t1 = time.time()
    train_x, train_y = apply_hog(trainset, size)
    val_x, val_y = apply_hog(valset, size)
    test_x, test_y = apply_hog(testset, size)
    clf.fit(train_x, train_y)
    clf.score(val_x, val_y)
    y_pred = clf.predict(test_x)
    t2 = time.time()
    print()
    print('Elapsed Time: {}'.format((t2 - t1)))
    print('Final accuracy: {} {}, {:.3f}'.format((name),(size),(accuracy_score(test_y, y_pred))))   
    print()

[LibSVM]
Elapsed Time: 5.300627946853638
Final accuracy: SVC 32, 0.484



KeyboardInterrupt: ignored